In [60]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier, XGBRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [61]:
# traveller_all.info()

# area_all = pd.read_csv('../csv/tn_visit_area_all.csv', low_memory=False)
# travel_all = pd.read_csv('../csv/tn_visit_travel_all.csv', low_memory=False)
# travelers_all = pd.read_csv('../csv/tn_visit_traveller_all.csv', low_memory=False)

In [62]:
# df = pd.merge(area_all, travel_all, how='left', on='TRAVEL_ID')
# df = pd.merge(df, travelers_all, how='left', on='TRAVELER_ID')


In [63]:
# df.to_csv('../csv/visit_all_in_one.csv', index=False)

df = pd.read_csv('../../csv/visit_all_in_one.csv')

C:\Users\user1\AppData\Local\Temp\ipykernel_15984\713656934.py:3: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../csv/visit_all_in_one.csv')


In [64]:
df_filter = df[~df['TRAVEL_MISSION_CHECK'].isnull()].copy()  # 'TRAVEL_MISSION_CHECK' 컬럼에 결측치가 없는 행만을 복사

# 미션 = 여행목적으로 보여짐 한 컬럼에 최대 3개가 있지만 일단 맨 처음 1개만 사용
df_filter.loc[:, 'TRAVEL_MISSION_INT'] = df_filter['TRAVEL_MISSION_CHECK'].str.split(';').str[0].astype(int)

In [95]:
df_learning = df_filter[[
    'GENDER',  # 성별
    'AGE_GRP',  # 연령대
    # 여행스타일
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6',
    'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1',  # 여행동기 (3까지 있음)
    'TRAVEL_COMPANIONS_NUM',  # 동반자 수
    'TRAVEL_MISSION_INT',  # 여행 목적 최우선순위
    'VISIT_AREA_NM',  # 방문지명
    'VISIT_AREA_TYPE_CD', # 방문지 유형 코드
    'DGSTFN',  # 만족도
]]

df_learning = df_learning.dropna()


In [97]:
# 변환할 열 목록
columns_to_convert = ['AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
                      'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
                      'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM']

# 성별도 int형으로 변환
df_learning['GENDER'] = df_learning['GENDER'].replace({'남': 1, '여': 0}).astype('int32')

# integer로 변환 (int32 사용으로 메모리 절약)
df_learning[columns_to_convert] = df_learning[columns_to_convert].astype('int32')

In [98]:
le = LabelEncoder()
df_learning['VISIT_AREA_NM_ENCODED'] = le.fit_transform(df_learning['VISIT_AREA_NM'])

features = ['GENDER', 'AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
            'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
            'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM', 'VISIT_AREA_NM_ENCODED']

rfr_X = df_learning[features]
rfr_y = df_learning['DGSTFN']

In [99]:
X_train_rfr, X_test_rfr, y_train_rfr, y_test_rfr = train_test_split(rfr_X, rfr_y, random_state=42)

In [108]:
sample_weights = np.where(y_train_rfr <= 3, 10, 1)
rfr = RandomForestRegressor()
rfr.fit(X_train_rfr, y_train_rfr, sample_weight=sample_weights)

RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=10,
                      random_state=42)

In [ ]:
# 평가
print("훈련 점수:", rfr.score(X_train_rfr, y_train_rfr))
print("테스트 점수:", rfr.score(X_test_rfr, y_test_rfr))

In [110]:
importances = rfr.feature_importances_
for feature, imp in zip(features, importances):
    print(f"{feature}: {imp:.4f}")

GENDER: 0.0175
AGE_GRP: 0.0466
TRAVEL_STYL_1: 0.0642
TRAVEL_STYL_2: 0.0670
TRAVEL_STYL_3: 0.0523
TRAVEL_STYL_4: 0.0664
TRAVEL_STYL_5: 0.0629
TRAVEL_STYL_6: 0.0650
TRAVEL_STYL_7: 0.0612
TRAVEL_STYL_8: 0.0677
TRAVEL_MOTIVE_1: 0.0679
TRAVEL_COMPANIONS_NUM: 0.0600
VISIT_AREA_TYPE_CD: 0.1012
VISIT_AREA_NM_ENCODED: 0.2001


In [75]:
features_xgbr = ['GENDER', 'AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
            'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
            'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM', 'VISIT_AREA_NM_ENCODED']

X_xgbr = df_learning[features_xgbr]
y_xgbr = df_learning['DGSTFN']

X_train_xgbr, X_test_xgbr, y_train_xgbr,  y_test_xgbr = train_test_split(X_xgbr, y_xgbr, test_size=0.2, random_state=42)

In [76]:
xgbr = XGBRegressor(objective='reg:squarederror', random_state=42)
xgbr.fit(X_train_xgbr, y_train_xgbr)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [77]:
# 특정 행 예측 비교 함수
def compare_regression_prediction(row_index):
    row = df_learning.iloc[row_index]
    actual_dgstfn = row['DGSTFN']  # 실제 만족도
    input_data = pd.DataFrame([row[features]], columns=features)

    # 예측
    pred_dgstfn = xgbr.predict(input_data)[0]

    # 결과 출력
    print(f"행 번호: {row_index}")
    print(f"방문지: {row['VISIT_AREA_NM']}")
    print(f"실제 만족도: {actual_dgstfn}")
    print(f"예측 만족도: {pred_dgstfn:.2f}")
    print(f"오차: {abs(actual_dgstfn - pred_dgstfn):.2f}")

In [78]:

for idx in [12789, 25846, 31599, 15451, 35656, 17458]:
    compare_regression_prediction(idx)
    print("-" * 50)

행 번호: 12789
방문지: 에이에이플레이스
실제 만족도: 4.0
예측 만족도: 3.85
오차: 0.15
--------------------------------------------------
행 번호: 25846
방문지: 국립경주박물관
실제 만족도: 5.0
예측 만족도: 4.13
오차: 0.87
--------------------------------------------------
행 번호: 31599
방문지: 금이야옥이야
실제 만족도: 4.0
예측 만족도: 4.13
오차: 0.13
--------------------------------------------------
행 번호: 15451
방문지: 벤투스풀빌라
실제 만족도: 4.0
예측 만족도: 4.37
오차: 0.37
--------------------------------------------------
행 번호: 35656
방문지: 센트럴시티오피스텔
실제 만족도: 5.0
예측 만족도: 4.57
오차: 0.43
--------------------------------------------------
행 번호: 17458
방문지: 동백섬
실제 만족도: 5.0
예측 만족도: 4.22
오차: 0.78
--------------------------------------------------


In [79]:
y_train_pred = xgbr.predict(X_train_xgbr)
y_test_pred = xgbr.predict(X_test_xgbr)

In [81]:
print("=== 훈련 데이터 성능 ===")
print(f"R² Score: {r2_score(y_train_xgbr, y_train_pred):.4f}")
print(f"MAE: {mean_absolute_error(y_train_xgbr, y_train_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train_xgbr, y_train_pred)):.4f}")

print("\n=== 테스트 데이터 성능 ===")
print(f"R² Score: {r2_score(y_test_xgbr, y_test_pred):.4f}")
print(f"MAE: {mean_absolute_error(y_test_xgbr, y_test_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test_xgbr, y_test_pred)):.4f}")

=== 훈련 데이터 성능 ===
R² Score: 0.2025
MAE: 0.5889
RMSE: 0.7567

=== 테스트 데이터 성능 ===
R² Score: 0.0577
MAE: 0.6420
RMSE: 0.8246
